***
# Optimize

***

In [4]:
import random
import numpy as np

In [2]:
#y(x1,x2)=5*x1^2+2*x2^2+3*x1-10*x2+4
def Func(ls):#函数
    return 5*ls[0]**2+2*ls[1]**2+3*ls[0]-10*ls[1]+4
def Grad(ls):#梯度
    g1=10*ls[0]+3
    g2=4*ls[1]-10
    return np.array([g1,g2])
def Hessian(ls):#黑塞矩阵
    return np.array([[10,0],[0,4]])

### 线性搜索

#### (1) 牛顿法

In [3]:
#newton法线性搜索
def NTsearch(x0,g,Grad,Hessian):#从x0点开始寻找g方向极小值点
    x1=x0.copy()
    x2=g#必须也在g方向上
    while np.linalg.norm(x2)>1e-3:
        x2=(g.dot(Grad(x1)))*g/np.dot(g,np.dot(Hessian(x1),g))
        x1-=x2
    return x1#返回更新的位置

#### (2) Goldstein线性搜索(非本人实现)

In [5]:
#Goldstein
def GSsearch(Func,Grad,g,x,lr,rho,t):#从x0点出发沿g方向搜索
    flag = 0
    a = 0
    b=lr
    fk = Func(x)
    gk = Grad(x)
    phi0 = fk
    dphi0 = np.dot(gk, g)
    alpha=b*random.uniform(0,1)
    while(flag==0):
        newfk = Func(x+alpha*g)
        phi = newfk
        if (phi - phi0 )<= (rho * alpha * dphi0):
            if (phi - phi0) >= ((1 - rho) * alpha * dphi0):
                flag = 1
            else:
                a = alpha
                b = b
                if (b <lr):
                    alpha = (a + b) / 2
                else:
                    alpha = t * alpha
        else:
            a = a
            b = alpha
            alpha = (a + b) / 2
    return alpha

#### (3) Wolfe线性搜索(非本人实现)

In [6]:
#Wolfe
def WFsearch(Func,Grad,g,x,alpham,rho,t):
  #σ∈(ρ,1)=0.75
  sigma=0.75
  flag = 0
 
  a = 0
  b = alpham
  fk = Func(x)
  gk = Grad(x)
 
  phi0 = fk
  dphi0 = np.dot(gk, g)
  alpha=b*random.uniform(0,1)
 
  while(flag==0):
    newfk = Func(x + alpha*g)
    phi = newfk
    if (phi - phi0 )<= (rho * alpha * dphi0):
      # if abs(np.dot(df(x + alpha * d),d))<=-sigma*dphi0:
      if (phi - phi0) >= ((1 - rho) * alpha * dphi0):
        flag = 1
      else:
        a = alpha
        b = b
        if (b < alpham):
          alpha = (a + b) / 2
        else:
          alpha = t * alpha
    else:
      a = a
      b = alpha
      alpha = (a + b) / 2
  return alpha

### 最优化方法

#### (1) 梯度下降法

In [7]:
#gradient descent
def GD(x0,Grad,lr=0.01):
	g=Grad(x0)#梯度
	x1=x0.copy()
	while np.linalg.norm(g)>1e-4:#梯度范数足够接近0则认为逼近极值点
		x1+=-lr*g#x=x-lr*grad
		g=Grad(x1)
	return x1

#### (2) 最速下降法

In [8]:
#steepest descent
def SD(x0,Grad,Hessian):
	g=-Grad(x0)
	x1=x0.copy()
	while np.linalg.norm(g)>1e-4:#解的精度
		#NewTon法线性搜索
		while True:
			lr=(g.dot(Grad(x1)))/np.dot(g,np.dot(Hessian(x1),g))#newton法迭代步长
			x2=lr*g
			x1-=x2#梯度下降
			if np.linalg.norm(x2)<1e-1:#线性搜索精度
				break
		g=-Grad(x1)
	return x1

#### (3) 共轭梯度法

In [9]:
#conjugate gradient
def CG(x0,Grad,Hessian):
	d=-Grad(x0)#搜索方向
	g=d
	x1=x0.copy()
	while np.linalg.norm(g)>1e-4:#解的精度
		#NewTon法线性搜索
		while True:
			lr=(d.dot(Grad(x1)))/np.dot(d,np.dot(Hessian(x1),d))#newton法迭代步长
			x2=lr*d
			x1-=x2#梯度下降
			if np.linalg.norm(x2)<1e-1:#线性搜索精度
				break
		#Wolfe线性搜索和Goldstein线性搜索
		#x1+=WFsearch(Func,Grad,g,x1,1,0.1,2)*g
		#x1+=GSsearch(Func,Grad,g,x1,1,0.1,2)*g
		g1=-Grad(x1)
		#系数b分两种更新方法:FR,PRP
		#b=g1.dot(g1)/g.dot(g)#FR
		b=(g1-g).dot(g1)/g.dot(g)#PRP
		d=g1+b*d
		'''可加
		if d.dot(g1)<0:
			d=g1
		'''
		g=g1
	return x1

#### (4) 牛顿法

In [10]:
#NewTon
def NewTon(x0,Grad,Hessian):
    x1=x0.copy()
    v=np.ones(len(x0))
    while np.linalg.norm(v)>1e-8:
        #gaussj消元法求解步长
        v=np.linalg.solve(Hessian(x1),-Grad(x1))#Hv=-grad
        x1+=v#更新
    return x1

#### (5) 拟牛顿法(未完全完成)

In [19]:
#BFGS
def BFGS(x0,Func,Grad):
    B=np.eye(x0.shape[0])
    g=Grad(x0)
    x1=x0.copy()
    while np.linalg.norm(g)>1e-4:
        s=-np.linalg.solve(B,g)
        #线性搜索
        lr=GSsearch(Func,Grad,s,x1,1,0.1,2)
        x1+=lr*s
        y=-g
        g=Grad(x1)
        y+=g
        B=B+y.T.dot(y)/y.dot(s)-(B.dot(s)).dot(s.dot(B))/s.dot(B.dot(s))
    return x1

### 测试

In [13]:
#初始点
x0=np.array([74.,-49.])

In [14]:
z=GD(x0,Grad)
print("极值点",z)
print("极值",Func(z))

极值点 [-0.3         2.49997587]
极值 -8.949999998835054


In [15]:
z=SD(x0,Grad,Hessian)
print("极值点",z)
print("极值",Func(z))

极值点 [-0.30000188  2.49998304]
极值 -8.949999999407265


In [16]:
z=CG(x0,Grad,Hessian)
print("极值点",z)
print("极值",Func(z))

极值点 [-0.3  2.5]
极值 -8.95


In [17]:
z=NewTon(x0,Grad,Hessian)
print("极值点",z)
print("极值",Func(z))

极值点 [-0.3  2.5]
极值 -8.950000000000001


### 效率

In [21]:
%timeit GD(x0,Grad)

6.64 ms ± 296 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [23]:
%timeit SD(x0,Grad,Hessian)

721 µs ± 106 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [24]:
%timeit CG(x0,Grad,Hessian)

208 µs ± 24.4 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [25]:
%timeit NewTon(x0,Grad,Hessian)

108 µs ± 9.07 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
